In [50]:
import os 
import pandas as pd
import clickhouse_connect

In [ ]:
#truy nhập vào click house và lấy dữ liệu từ bảng tương ứng với điều kiện cụ thể
client = clickhouse_connect.get_client(
    host=os.getenv("CLICKHOUSE_HOST"),
    port=int(os.getenv("CLICKHOUSE_PORT")),
    username=os.getenv("CLICKHOUSE_USER"),
    password=os.getenv("CLICKHOUSE_PASSWORD")
)
query = """
SELECT *
FROM cem_network_test.result_full_flow_2025_31_12
WHERE delay_init_up > 0 and delay_init_up < 1500
"""
df = client.query_df(query)

In [52]:
# cắt dữ liệu theo từng 30p 
df['event_time'] = pd.to_datetime(df['from_time'], unit='s')
df['time_window'] = df['event_time'].dt.floor('30T')

C:\Users\This PC\AppData\Local\Temp\ipykernel_19132\2609242498.py:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df['time_window'] = df['event_time'].dt.floor('30T')


In [53]:
#đếm số lượng lỗi xem flow này có lỗi hay k 
FAIL_CODE=[1]   
df['is_fail'] = df['cs_cause_code'].isin(FAIL_CODE)
df.head()

,flow_idx,ts_idx,client_mac,client_ip,server_ip,proto,client_port,server_port,tls_domain,dns_domain,delay_init_up,delay_init_down,from_time,to_time,traffic_up,traffic_down,cs_cause_code,event_time,time_window,is_fail
0,2360488914,1767147096,a4:f4:c2:dd:ef:04,2001:ee0:43a1:18a0:cca1:5aa1:3c8:641f,2402:9d80:100:0:0:0:0:61a,6,59499,443,pca-photo-stal-7.zdn.vn,pca-photo-stal-7.zdn.vn,2,4,1767147096,1767147100,13,278,0,2025-12-31 02:11:36,2025-12-31 02:00:00,False
1,2360462373,1767147089,b8:29:03:e1:71:59,2001:ee0:4390:cb30:f585:f43c:da:1282,2404:6800:4005:805:0:0:0:2001,256,57375,443,ccp-lh.googleusercontent.com,_,22,4,1767147089,1767147089,14,197,-2,2025-12-31 02:11:29,2025-12-31 02:00:00,False
2,2360454900,1767147087,4c:12:e8:64:ea:c1,2001:ee0:43a7:20:e849:1397:f1b5:6900,2001:ee0:43c8:f340:c9d3:3729:199b:9c2f,6,26515,57760,_,_,2,2,1767147087,1767147087,0,0,-4,2025-12-31 02:11:27,2025-12-31 02:00:00,False
3,2360472135,1767147091,b8:29:03:2a:35:99,2001:ee0:43a1:4a00:70bc:4d36:c56f:31b0,2404:6800:4005:81e:0:0:0:2003,6,54605,443,connectivitycheck.gstatic.com,_,23,9,1767147091,1767147091,1,3,0,2025-12-31 02:11:31,2025-12-31 02:00:00,False
4,2360465202,1767147089,30:4f:75:82:cb:e9,2001:ee0:4391:2b40:b88f:1fc3:3221:c968,2001:ee0:3240:100:0:0:71ab:e609,6,51927,443,bag.itunes.apple.com,_,1,5,1767147089,1767147090,2,7,0,2025-12-31 02:11:29,2025-12-31 02:00:00,False


In [54]:
# tính toán dữ liệu dựa trên các nhóm server_ip và time_window
agg = (
    df
    .groupby(['time_window', 'server_ip'])
    .agg(
        total_flow=('flow_idx', 'count'),
        total_failures=('is_fail', 'sum'),
        distinct_clients=('client_ip', 'nunique'),
        p_20_delayup=('delay_init_up', lambda x: x.quantile(0.2)),
        p_50_delayup=('delay_init_up', lambda x: x.quantile(0.5)),
        p_90_delayup=('delay_init_up', lambda x: x.quantile(0.9)),
    )
).reset_index()

agg['failure_rate'] = agg['total_failures'] / agg['total_flow']


In [55]:
agg.head()

,time_window,server_ip,total_flow,total_failures,distinct_clients,p_20_delayup,p_50_delayup,p_90_delayup,failure_rate
0,2025-12-30 16:30:00,1.0.0.1,5,0,3,32.0,33.0,40.0,0.0
1,2025-12-30 16:30:00,1.1.1.1,37,0,3,34.0,35.0,37.0,0.0
2,2025-12-30 16:30:00,1.53.117.216,6,0,1,4.0,4.0,4.0,0.0
3,2025-12-30 16:30:00,1.53.117.228,4,0,1,5.6,6.0,6.0,0.0
4,2025-12-30 16:30:00,1.92.66.95,1,0,1,546.0,546.0,546.0,0.0


In [56]:
#chuẩn hóa dữ liệu trước khi lưu vào clickhouse
agg = agg.astype({
    'total_flow': 'int',
    'total_failures': 'int',
    'distinct_clients': 'int',
    'p_20_delayup': 'float',
    'p_50_delayup': 'float',
    'p_90_delayup': 'float',
    'failure_rate': 'float'
})

In [57]:
agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180501 entries, 0 to 180500
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   time_window       180501 non-null  datetime64[ns]
 1   server_ip         180501 non-null  string        
 2   total_flow        180501 non-null  int64         
 3   total_failures    180501 non-null  int64         
 4   distinct_clients  180501 non-null  int64         
 5   p_20_delayup      180501 non-null  float64       
 6   p_50_delayup      180501 non-null  float64       
 7   p_90_delayup      180501 non-null  float64       
 8   failure_rate      180501 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(3), string(1)
memory usage: 12.4 MB


In [58]:
#lưu dữ liệu vào database clickhouse
client.insert_df(
    table='cem_kpi.agg_server_30m_test',
    df=agg
)
